# Defining libraries

In [24]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense


# Cleaning data of corrupted images

In [39]:
import os

num_skipped = 0
for folder_name in ("COVID-19", "Normal"):
    folder_path = os.path.join("D://DeepLearningPractice/COVID-Xray/TrainImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 56 images


# Data preprocessing

In [40]:
#Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
rescale=1./255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(
'D://DeepLearningPractice/COVID-Xray/TrainImages',
target_size=(64, 64),
batch_size=32,
class_mode='binary')

test_set = test_datagen.flow_from_directory(
    'D://DeepLearningPractice/COVID-Xray/TestImages',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')
        

Found 472 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [57]:
classes = training_set.class_indices
print(classes)

{'COVID-19': 0, 'Normal': 1}


# CNN model

In [41]:
model = tf.keras.Sequential()

# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=3, activation='relu', input_shape=(64,64,3))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 29, 29, 16)        2320      
_________________________________________________________________
flatten_12 (Flatten)         (None, 13456)             0         
_________________________________________________________________
dense_21 (Dense)             (None, 128)               1722496   
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 129       
Total params: 1,725,393
Trainable params: 1,725,393
Non-trainable params: 0
___________________________________________

# Creating Model

In [42]:
model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(
training_set,
epochs=25,
validation_data=test_set)

Epoch 1/25
15/15 [==============================] - 25s 2s/step - loss: 0.5885 - accuracy: 0.7076 - val_loss: 0.3147 - val_accuracy: 0.9000
Epoch 2/25
15/15 [==============================] - 17s 1s/step - loss: 0.2627 - accuracy: 0.8983 - val_loss: 0.2255 - val_accuracy: 0.9083
Epoch 3/25
15/15 [==============================] - 17s 1s/step - loss: 0.1630 - accuracy: 0.9343 - val_loss: 0.1890 - val_accuracy: 0.9167
Epoch 4/25
15/15 [==============================] - 18s 1s/step - loss: 0.1718 - accuracy: 0.9364 - val_loss: 0.1911 - val_accuracy: 0.8917
Epoch 5/25
15/15 [==============================] - 17s 1s/step - loss: 0.1334 - accuracy: 0.9470 - val_loss: 0.1511 - val_accuracy: 0.9167
Epoch 6/25
15/15 [==============================] - 24s 2s/step - loss: 0.1627 - accuracy: 0.9301 - val_loss: 0.2911 - val_accuracy: 0.8750
Epoch 7/25
15/15 [==============================] - 19s 1s/step - loss: 0.1151 - accuracy: 0.9534 - val_loss: 0.1758 - val_accuracy: 0.9250
Epoch 8/25
15/15 [==

# Model evaluation

In [45]:
score = model.evaluate(test_set, verbose=0)


# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.949999988079071
